In [51]:
# https://github.com/h2oai/h2o-tutorials/blob/master/tutorials/gbm-randomforest/GBM_RandomForest_Example.py
import h2o
import os

h2o.init()
#df = pd.read_csv("Modelar_UH2020.txt", sep = "|")
df = h2o.import_file(os.path.realpath("Modelar_UH2020.txt"), sep = "|")


train, valid, test = df.split_frame([0.6, 0.2], seed=1234)
df_X = df.col_names[:-1]     #last column is Cover_Type, our desired response variable 
df_y = df.col_names[-1]  

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.211-b12, mixed mode)
  Starting server from C:\Users\pablo.jordan.de.la.1\Anaconda\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\PABLOJ~1.1\AppData\Local\Temp\tmpibo4xw41
  JVM stdout: C:\Users\PABLOJ~1.1\AppData\Local\Temp\tmpibo4xw41\h2o_pablo_jordan_de_la_1_started_from_python.out
  JVM stderr: C:\Users\PABLOJ~1.1\AppData\Local\Temp\tmpibo4xw41\h2o_pablo_jordan_de_la_1_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,1 month
H2O cluster name:,H2O_from_python_pablo_jordan_de_la_1_g1cxst
H2O cluster total nodes:,1
H2O cluster free memory:,1.752 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [52]:
rf_v1 = H2ORandomForestEstimator(
    model_id="rf_covType_v1",
    ntrees=200,
    stopping_rounds=2,
    score_each_iteration=True,
    seed=1000000)

rf_v1.train(df_X, df_y, training_frame=train, validation_frame=valid)


#### GBM
gbm_v1 = H2OGradientBoostingEstimator(
    model_id="gbm_covType_v1",
    seed=2000000
)
gbm_v1.train(df_X, df_y, training_frame=train, validation_frame=valid)

gbm_v2 = H2OGradientBoostingEstimator(
    ntrees=20,
    learn_rate=0.2,
    max_depth=10,
    stopping_tolerance=0.01, #10-fold increase in threshold as defined in rf_v1
    stopping_rounds=2,
    score_each_iteration=True,
    model_id="gbm_covType_v2",
    seed=2000000
)
gbm_v2.train(df_X, df_y, training_frame=train, validation_frame=valid)

gbm_v3 = H2OGradientBoostingEstimator(
    ntrees=30,
    learn_rate=0.3,
    max_depth=10,
    sample_rate=0.7,
    col_sample_rate=0.7,
    stopping_rounds=2,
    stopping_tolerance=0.01, #10-fold increase in threshold as defined in rf_v1
    score_each_iteration=True,
    model_id="gbm_covType_v3",
    seed=2000000
)
gbm_v3.train(df_X, df_y, training_frame=train, validation_frame=valid)

rf_v2 = H2ORandomForestEstimator(
    model_id="rf_covType_v2",
    ntrees=200,
    max_depth=30,
    stopping_rounds=2,
    stopping_tolerance=0.01,
    score_each_iteration=True,
    seed=3000000)
rf_v2.train(df_X, df_y, training_frame=train, validation_frame=valid)

final_rf_predictions = rf_v2.predict(test[:-1])
final_rf_predictions
rf_v2.hit_ratio_table(valid=True)

#test set accuracy
(final_rf_predictions['predict']==test['CLASE']).as_data_frame(use_pandas=True).mean()


#h2o.shutdown(prompt=False)


drf Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


predict    0.915579
dtype: float64

In [57]:
rf_v1
#rf_v1.score_history()
#rf_v1.hit_ratio_table(valid=True)

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  rf_covType_v1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,52.0,364.0,5132091.0,15.0,20.0,19.85989,109.0,2421.0,1117.2968




ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.0893239866677077
RMSE: 0.2988711874164315
LogLoss: 0.41261195247035887
Mean Per-Class Error: 0.5930571316251972

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,AGRICULTURE,INDUSTRIAL,OFFICE,OTHER,PUBLIC,RESIDENTIAL,RETAIL,Error,Rate
0,109.0,27.0,0.0,0.0,2.0,54.0,3.0,0.441026,86 / 195
1,19.0,1535.0,60.0,15.0,47.0,959.0,25.0,0.422932,1.125 / 2.660
2,0.0,180.0,121.0,6.0,40.0,755.0,8.0,0.890991,989 / 1.110
3,0.0,29.0,6.0,257.0,62.0,438.0,5.0,0.677541,540 / 797
4,8.0,58.0,23.0,71.0,319.0,1360.0,13.0,0.827754,1.533 / 1.852
5,12.0,104.0,22.0,42.0,102.0,53764.0,11.0,0.005420,293 / 54.057
6,5.0,62.0,26.0,8.0,41.0,1013.0,149.0,0.885736,1.155 / 1.304
7,153.0,1995.0,258.0,399.0,613.0,58343.0,214.0,0.092311,5.721 / 61.975



Top-7 Hit Ratios: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
0,,2020-03-05 19:48:03,0.147 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-03-05 19:48:06,2.947 sec,1.0,0.339136,2.647456,0.123635,0.336381,2.672626,0.122345
2,,2020-03-05 19:48:09,6.092 sec,2.0,0.334161,2.470192,0.118941,0.309478,1.524462,0.102513
3,,2020-03-05 19:48:10,7.073 sec,3.0,0.330373,2.265348,0.116385,0.303904,1.114521,0.096777
4,,2020-03-05 19:48:11,8.173 sec,4.0,0.324943,1.975868,0.111844,0.301241,0.848663,0.095173
5,,2020-03-05 19:48:12,9.184 sec,5.0,0.320736,1.778715,0.109042,0.299484,0.729053,0.093326
6,,2020-03-05 19:48:13,10.391 sec,6.0,0.317883,1.618157,0.106365,0.298333,0.669824,0.092014
7,,2020-03-05 19:48:14,11.342 sec,7.0,0.315142,1.411140,0.104380,0.297250,0.586302,0.091333
8,,2020-03-05 19:48:15,12.523 sec,8.0,0.313484,1.325572,0.102721,0.296676,0.567673,0.090507
9,,2020-03-05 19:48:16,13.573 sec,9.0,0.311756,1.221912,0.101229,0.296407,0.544314,0.090847



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,GEOM_R2,33314.902344,1.000000,0.103873
1,GEOM_R3,31593.650391,0.948334,0.098506
2,AREA,29419.566406,0.883075,0.091728
3,GEOM_R1,23281.410156,0.698829,0.072589
4,MAXBUILDINGFLOOR,19065.521484,0.572282,0.059445
5,CONTRUCTIONYEAR,16459.890625,0.494070,0.051320
6,CADASTRALQUALITYID,16255.796875,0.487944,0.050684
7,GEOM_R4,13891.293945,0.416969,0.043312
8,X,12897.797852,0.387148,0.040214
9,Y,11526.734375,0.345993,0.035939



See the whole table with table.as_data_frame()


In [58]:
gbm_v1
#gbm_v1.score_history()
#gbm_v1.hit_ratio_table(valid=True)

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  gbm_covType_v1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,350.0,129696.0,5.0,5.0,5.0,9.0,32.0,24.834286




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.0811798610835376
RMSE: 0.2849207979132755
LogLoss: 0.31423748486818126
Mean Per-Class Error: 0.5123943303406857

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,AGRICULTURE,INDUSTRIAL,OFFICE,OTHER,PUBLIC,RESIDENTIAL,RETAIL,Error,Rate
0,145.0,11.0,1.0,0.0,0.0,38.0,0.0,0.256410,50 / 195
1,5.0,1620.0,49.0,11.0,35.0,917.0,23.0,0.390977,1.040 / 2.660
2,0.0,143.0,207.0,2.0,26.0,727.0,5.0,0.813514,903 / 1.110
3,0.0,24.0,4.0,361.0,25.0,379.0,4.0,0.547051,436 / 797
4,3.0,59.0,15.0,39.0,506.0,1224.0,6.0,0.726782,1.346 / 1.852
5,7.0,81.0,12.0,28.0,77.0,53848.0,4.0,0.003866,209 / 54.057
6,3.0,73.0,13.0,3.0,41.0,973.0,198.0,0.848160,1.106 / 1.304
7,163.0,2011.0,301.0,444.0,710.0,58106.0,240.0,0.082130,5.090 / 61.975



Top-7 Hit Ratios: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
0,,2020-03-05 19:49:34,0.127 sec,0.0,0.857143,1.945910,0.236999,0.857143,1.945910,0.232198
1,,2020-03-05 19:49:35,1.716 sec,1.0,0.773668,1.497249,0.103881,0.773578,1.496958,0.104798
2,,2020-03-05 19:49:36,2.831 sec,2.0,0.703756,1.240861,0.102509,0.703661,1.240886,0.103825
3,,2020-03-05 19:49:37,3.744 sec,3.0,0.642548,1.063056,0.101864,0.642398,1.063096,0.103339
4,,2020-03-05 19:49:42,8.064 sec,9.0,0.413773,0.578900,0.098669,0.414742,0.582342,0.100909
5,,2020-03-05 19:49:46,12.442 sec,15.0,0.336641,0.434965,0.094877,0.339579,0.442378,0.098138
6,,2020-03-05 19:49:50,16.829 sec,21.0,0.312095,0.381474,0.092166,0.316926,0.393670,0.096388
7,,2020-03-05 19:49:55,21.257 sec,27.0,0.301468,0.355850,0.090069,0.308512,0.374349,0.095027
8,,2020-03-05 19:49:59,25.717 sec,33.0,0.295152,0.339596,0.087455,0.304349,0.363567,0.093278
9,,2020-03-05 19:50:04,30.124 sec,39.0,0.290642,0.328112,0.084825,0.301829,0.356898,0.091868



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,AREA,6010.665039,1.000000,0.245549
1,GEOM_R2,2506.568848,0.417020,0.102399
2,CADASTRALQUALITYID,1753.878662,0.291794,0.071650
3,MAXBUILDINGFLOOR,1713.393921,0.285059,0.069996
4,Q_B_2_0_9,1498.393799,0.249289,0.061213
5,X,1332.817749,0.221742,0.054449
6,GEOM_R1,1108.954224,0.184498,0.045303
7,Y,951.803345,0.158352,0.038883
8,GEOM_R3,919.739807,0.153018,0.037573
9,Q_B_2_0_2,585.801331,0.097460,0.023931



See the whole table with table.as_data_frame()


In [59]:
gbm_v2
#gbm_v2.score_history()
#gbm_v2.hit_ratio_table(valid=True)

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  gbm_covType_v2


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,17.0,119.0,335122.0,10.0,10.0,10.0,39.0,409.0,219.63866




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.04640769813521373
RMSE: 0.21542446039206814
LogLoss: 0.18538353480481953
Mean Per-Class Error: 0.2947512962161074

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,AGRICULTURE,INDUSTRIAL,OFFICE,OTHER,PUBLIC,RESIDENTIAL,RETAIL,Error,Rate
0,176.0,4.0,0.0,0.0,0.0,15.0,0.0,0.097436,19 / 195
1,3.0,2015.0,13.0,5.0,16.0,592.0,16.0,0.242481,645 / 2.660
2,0.0,57.0,613.0,0.0,4.0,431.0,5.0,0.447748,497 / 1.110
3,0.0,11.0,0.0,553.0,5.0,225.0,3.0,0.306148,244 / 797
4,1.0,15.0,3.0,5.0,1137.0,687.0,4.0,0.386069,715 / 1.852
5,1.0,16.0,2.0,5.0,2.0,54027.0,4.0,0.000555,30 / 54.057
6,1.0,30.0,3.0,0.0,10.0,716.0,544.0,0.582822,760 / 1.304
7,182.0,2148.0,634.0,568.0,1174.0,56693.0,576.0,0.046954,2.910 / 61.975



Top-7 Hit Ratios: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
0,,2020-03-05 19:50:12,0.298 sec,0.0,0.857143,1.945910,0.236999,0.857143,1.945910,0.232198
1,,2020-03-05 19:50:14,1.912 sec,1.0,0.650643,1.078169,0.084808,0.653980,1.093111,0.097701
2,,2020-03-05 19:50:16,3.507 sec,2.0,0.530847,0.790115,0.079823,0.538577,0.816903,0.095757
3,,2020-03-05 19:50:17,5.134 sec,3.0,0.443930,0.618506,0.077354,0.456797,0.655528,0.094298
4,,2020-03-05 19:50:19,6.682 sec,4.0,0.382412,0.506854,0.074401,0.400254,0.552117,0.093472
5,,2020-03-05 19:50:20,8.257 sec,5.0,0.339345,0.429585,0.071642,0.362940,0.484369,0.092646
6,,2020-03-05 19:50:22,9.761 sec,6.0,0.309786,0.375220,0.069835,0.338619,0.438612,0.093083
7,,2020-03-05 19:50:23,11.274 sec,7.0,0.289105,0.334726,0.067672,0.322994,0.406606,0.092014
8,,2020-03-05 19:50:25,12.962 sec,8.0,0.274015,0.303853,0.065010,0.313097,0.385225,0.091479
9,,2020-03-05 19:50:27,14.501 sec,9.0,0.262269,0.278582,0.062348,0.306784,0.370397,0.091090



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,AREA,3473.147461,1.000000,0.189971
1,GEOM_R2,1720.944580,0.495500,0.094131
2,MAXBUILDINGFLOOR,1311.757080,0.377685,0.071749
3,CADASTRALQUALITYID,1175.634888,0.338493,0.064304
4,GEOM_R1,1087.445557,0.313101,0.059480
5,X,1009.740845,0.290728,0.055230
6,CONTRUCTIONYEAR,890.689392,0.256450,0.048718
7,GEOM_R3,843.887390,0.242975,0.046158
8,Q_B_2_0_9,843.444641,0.242847,0.046134
9,Y,692.725037,0.199452,0.037890



See the whole table with table.as_data_frame()


In [60]:
gbm_v3
#gbm_v3.score_history()
#gbm_v3.hit_ratio_table(valid=True)

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  gbm_covType_v3


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,12.0,84.0,184600.0,10.0,10.0,10.0,31.0,344.0,170.38095




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.05698768028632506
RMSE: 0.2387209255308907
LogLoss: 0.22070484209353003
Mean Per-Class Error: 0.36453910248812027

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,AGRICULTURE,INDUSTRIAL,OFFICE,OTHER,PUBLIC,RESIDENTIAL,RETAIL,Error,Rate
0,161.0,8.0,1.0,0.0,1.0,21.0,3.0,0.174359,34 / 195
1,7.0,1907.0,38.0,5.0,32.0,644.0,27.0,0.283083,753 / 2.660
2,0.0,100.0,470.0,0.0,16.0,517.0,7.0,0.576577,640 / 1.110
3,0.0,15.0,3.0,498.0,25.0,252.0,4.0,0.375157,299 / 797
4,1.0,35.0,13.0,17.0,932.0,843.0,11.0,0.496760,920 / 1.852
5,18.0,48.0,14.0,22.0,49.0,53884.0,22.0,0.003200,173 / 54.057
6,2.0,39.0,15.0,6.0,33.0,743.0,466.0,0.642638,838 / 1.304
7,189.0,2152.0,554.0,548.0,1088.0,56904.0,540.0,0.059008,3.657 / 61.975



Top-7 Hit Ratios: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
0,,2020-03-05 19:50:40,0.127 sec,0.0,0.857143,1.945910,0.236999,0.857143,1.945910,0.232198
1,,2020-03-05 19:50:41,1.381 sec,1.0,0.527465,0.792504,0.093312,0.531307,0.806701,0.100617
2,,2020-03-05 19:50:42,2.337 sec,2.0,0.410647,0.564285,0.086390,0.420055,0.591150,0.096729
3,,2020-03-05 19:50:43,3.280 sec,3.0,0.345215,0.443397,0.082372,0.360679,0.481936,0.095076
4,,2020-03-05 19:50:44,4.235 sec,4.0,0.309026,0.373364,0.078725,0.330611,0.424159,0.094250
5,,2020-03-05 19:50:45,5.179 sec,5.0,0.289035,0.331047,0.076095,0.315187,0.391522,0.093812
6,,2020-03-05 19:50:46,6.115 sec,6.0,0.276197,0.301831,0.072642,0.307073,0.371627,0.092646
7,,2020-03-05 19:50:47,7.048 sec,7.0,0.266720,0.280671,0.070044,0.302868,0.361707,0.092646
8,,2020-03-05 19:50:48,7.991 sec,8.0,0.259144,0.261468,0.066640,0.300734,0.354952,0.091917
9,,2020-03-05 19:50:49,8.891 sec,9.0,0.252953,0.248195,0.064655,0.299483,0.352189,0.093132



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,AREA,2220.880859,1.000000,0.219418
1,GEOM_R2,817.474670,0.368086,0.080765
2,MAXBUILDINGFLOOR,645.672852,0.290728,0.063791
3,GEOM_R1,643.481689,0.289742,0.063575
4,CADASTRALQUALITYID,544.212646,0.245044,0.053767
5,X,520.026611,0.234153,0.051378
6,GEOM_R3,504.381744,0.227109,0.049832
7,CONTRUCTIONYEAR,410.976227,0.185051,0.040604
8,Q_B_2_0_9,399.129944,0.179717,0.039433
9,Y,317.061157,0.142764,0.031325



See the whole table with table.as_data_frame()


In [61]:
rf_v2
#rf_v2.score_history()
#rf_v2.hit_ratio_table(valid=True)

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  rf_covType_v2


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,34.0,238.0,4611642.0,16.0,30.0,27.760504,104.0,4046.0,1536.9622




ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.08985862217882419
RMSE: 0.2997642776896944
LogLoss: 0.6123638181215345
Mean Per-Class Error: 0.5828422088311199

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,AGRICULTURE,INDUSTRIAL,OFFICE,OTHER,PUBLIC,RESIDENTIAL,RETAIL,Error,Rate
0,112.0,27.0,2.0,0.0,6.0,45.0,3.0,0.425641,83 / 195
1,18.0,1537.0,70.0,16.0,64.0,917.0,38.0,0.422180,1.123 / 2.660
2,0.0,184.0,127.0,5.0,40.0,741.0,13.0,0.885586,983 / 1.110
3,0.0,27.0,11.0,269.0,72.0,411.0,7.0,0.662484,528 / 797
4,5.0,71.0,36.0,63.0,333.0,1327.0,17.0,0.820194,1.519 / 1.852
5,13.0,131.0,34.0,48.0,137.0,53667.0,27.0,0.007215,390 / 54.057
6,4.0,73.0,25.0,8.0,45.0,962.0,187.0,0.856595,1.117 / 1.304
7,152.0,2050.0,305.0,409.0,697.0,58070.0,292.0,0.092666,5.743 / 61.975



Top-7 Hit Ratios: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
0,,2020-03-05 19:50:52,0.002 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-03-05 19:50:53,1.092 sec,1.0,0.367663,3.849040,0.127773,0.357457,3.570506,0.123366
2,,2020-03-05 19:50:54,2.135 sec,2.0,0.355260,3.348111,0.124903,0.313980,1.925783,0.103339
3,,2020-03-05 19:50:55,3.236 sec,3.0,0.345731,3.017469,0.119468,0.305878,1.505839,0.096048
4,,2020-03-05 19:50:57,4.318 sec,4.0,0.337080,2.700500,0.114318,0.302070,1.259152,0.093375
5,,2020-03-05 19:50:58,5.459 sec,5.0,0.330088,2.403884,0.111057,0.299338,1.067570,0.091965
6,,2020-03-05 19:50:59,6.630 sec,6.0,0.325252,2.191245,0.108104,0.297739,0.944815,0.091187
7,,2020-03-05 19:51:00,7.872 sec,7.0,0.321212,1.997984,0.105627,0.297451,0.874498,0.090896
8,,2020-03-05 19:51:01,9.160 sec,8.0,0.317515,1.828187,0.103475,0.297083,0.800170,0.090993
9,,2020-03-05 19:51:03,10.382 sec,9.0,0.313794,1.652526,0.101776,0.296433,0.735443,0.089681



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,GEOM_R2,26840.976562,1.000000,0.108544
1,AREA,25827.281250,0.962233,0.104445
2,GEOM_R3,20714.921875,0.771765,0.083771
3,GEOM_R1,19677.314453,0.733107,0.079575
4,MAXBUILDINGFLOOR,15281.729492,0.569343,0.061799
5,CONTRUCTIONYEAR,14062.763672,0.523929,0.056870
6,GEOM_R4,12458.070312,0.464144,0.050380
7,CADASTRALQUALITYID,12334.083008,0.459524,0.049879
8,X,10884.134766,0.405504,0.044015
9,Y,10033.136719,0.373799,0.040574



See the whole table with table.as_data_frame()


In [62]:
final_rf_predictions.as_data_frame()
#rf_v2.hit_ratio_table(valid=True)

,predict,AGRICULTURE,INDUSTRIAL,OFFICE,OTHER,PUBLIC,RESIDENTIAL,RETAIL
0,RESIDENTIAL,0.000000,0.000000,0.000000,0.000000,0.000925,0.999075,0.000000
1,INDUSTRIAL,0.001142,0.955669,0.001889,0.000000,0.001160,0.029109,0.011031
2,RESIDENTIAL,0.000000,0.002110,0.000000,0.000083,0.006124,0.991683,0.000000
3,RESIDENTIAL,0.000000,0.001831,0.000000,0.001195,0.001355,0.994172,0.001447
4,RESIDENTIAL,0.000000,0.001172,0.011440,0.000071,0.002951,0.979643,0.004723
5,RESIDENTIAL,0.000000,0.001417,0.000000,0.000000,0.003422,0.995029,0.000132
6,RESIDENTIAL,0.000000,0.006664,0.002418,0.001257,0.000816,0.985825,0.003020
7,RESIDENTIAL,0.000000,0.000000,0.000272,0.007226,0.142539,0.786445,0.063518
8,RESIDENTIAL,0.000000,0.004805,0.000000,0.009280,0.001946,0.981251,0.002719
9,RESIDENTIAL,0.000000,0.003454,0.013180,0.000000,0.004645,0.969852,0.008870
